# Project Part 1: Identify the Blitz

A defensive blitz is when 5+ defensive players rush into the offensive zone. This notebook identifies the blitz on each play.

In its current state, each week must be run by altering the second chunk of code with the line denoting `weekNum`. Future adjustments would loop through each file so that the notebook would only be run a single time. 

In [1]:
import pandas as pd
import numpy as np
from ipynb.fs.full.Functions import *       # import functions from Functions.ipynb
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'

In [42]:
# read in csv files 

#### URGENT:  what week is it? ###
weekNum = 8
filename = 'week' + str(weekNum) + '.csv'

week = pd.read_csv(filename)

In [12]:
# cont. read csv files, these are the same regardless of week
players = pd.read_csv('players.csv')
all_plays = pd.read_csv('plays.csv')

In [43]:
# subset all_plays to include only those from week in question
plays_wk = all_plays[all_plays['gameId'].isin(week['gameId'])]
# create column where we will assign a blitz, 1 (or not, 0)
plays_wk['numBackfield'] = 0
plays_wk['isBlitz'] = 0
plays_wk['weekNum'] = weekNum

In [44]:
# get gameIds of the games for the week
gameIdAll = plays_wk['gameId'].unique()

In [45]:
for game in gameIdAll:      # loop through all games
    # get the plays associated with this particular game
    playIdAll = plays_wk.loc[plays_wk['gameId'] == game, 'playId'].unique()
    for play in playIdAll:  # loop through all plays of this game
        ## SET UP PLAY AND ROSTER ##
        # get offense and defense
        offenseS = plays_wk.loc[(plays_wk['gameId'] == game) & (plays_wk['playId'] == play), 'possessionTeam']#.iloc[0]
        offense = offenseS.iloc[0]
        defenseS = plays_wk.loc[(plays_wk['gameId'] == game) & (plays_wk['playId'] == play), 'defensiveTeam']#.iloc[0]
        defense = defenseS.iloc[0]

        # store the current play in its own df
        currentPlay = week.loc[(week['gameId']==game) & (week['playId']==play)]
        
        # who is in this play?
        roster_play = players[players['nflId'].isin(currentPlay['nflId'])]
        # who is the qb of this play?
        qb_id = roster_play.loc[roster_play['officialPosition'] == 'QB'].nflId.tolist()[0]
        
        ## IDENTIFY OFFENSIVE ZONE ##
        ## The offensive zone is defined as a rectangle that spans the width of the field and begins at the offensive line ##
        backfield_df = def_off_back(currentPlay, offense) 
        
        ## EVALUTE THE PLAY FOR A BLITZ ##
        ## Here, we check if 5+ defensive plays enter the offensive zone. If there are 5+, this is a blitz. ##
        # get defensive players
        D_play = currentPlay.loc[currentPlay.team == defense]
        D_play = D_play.sort_values(by='frameId')
        
        # for plotting purposes, get everyone else too
        O_play = currentPlay.loc[(currentPlay['nflId'] != qb_id) & (currentPlay['team']!=defense)]
        O_play = O_play.sort_values(by='frameId')
        
        # get the same information, but for QB only
        QB_play = currentPlay.loc[currentPlay['nflId']==qb_id]
        QB_play = QB_play.sort_values(by='frameId')
        
        # get the same information, but for the ball
        ball_play = currentPlay.loc[currentPlay['team']=='football']
        ball_play = ball_play.sort_values(by='frameId')
        
        # what direction is play? defensive will move opposite
        # if play direction is 'right', we are looking for x values to get smaller
        # if play direction is 'left', we are looking for x values to get bigger
        play_direction = D_play.playDirection.iloc[0]
        
    
        # this could be a function... edit later

        # do any defensive players cross into the backfield (the answer will be yes)
        # get the frame that they FIRST step into the backfield

        # still like jerseyNumber here over nflId bc that is what will be used in animation + more familiar to readers/viewers

        D_jerseys = D_play.jerseyNumber.unique()
        firsts = pd.DataFrame(columns=["jerseyNumber", "backfield_frame"]) # hold these first values in a dataframe
        for player in D_jerseys: # for each player...  make sure to add an edit for R/L later </>
            if play_direction == 'right':
                # check: do they cross into the offensive backfield?
                if (D_play[(D_play.jerseyNumber == player) & (D_play.x < backfield_df.x.loc[0])]).empty == False: 
                    # if a player has x-coordinates that are located in the offensive backfield, 
                    # print(f'player jersey #{player} has crossed into offensive backfield')
                    # append his jerseyNumber and the frameId where he crossed to the dataframe `firsts`
                    frame = (D_play[(D_play.jerseyNumber == player) & (D_play.x < backfield_df.x.loc[0])]).frameId.iloc[0]
                    new_entry = pd.DataFrame({'jerseyNumber': player, 'backfield_frame': frame}, index=[0])
                    firsts = pd.concat([new_entry,firsts.loc[:]]).reset_index(drop=True)       
                else:
                    # else, a player does NOT have any x-coordinates in the offensive backfield, 
                    # print(f'player jersey #{player} DID NOT cross into offensive backfield')
                    pass
            else: # play_direction == 'left'
                # check: do they cross into the offensive backfield?
                if (D_play[(D_play.jerseyNumber == player) & (D_play.x > backfield_df.x.loc[0])]).empty == False: 
                    # if a player has x-coordinates that are located in the offensive backfield, 
                    # print(f'player jersey #{player} has crossed into offensive backfield')
                    # append his jerseyNumber and the frameId where he crossed to the dataframe `firsts`
                    frame = (D_play[(D_play.jerseyNumber == player) & (D_play.x > backfield_df.x.loc[0])]).frameId.iloc[0]
                    new_entry = pd.DataFrame({'jerseyNumber': player, 'backfield_frame': frame}, index=[0])
                    firsts = pd.concat([new_entry,firsts.loc[:]]).reset_index(drop=True)       
                else:
                    # else, a player does NOT have any x-coordinates in the offensive backfield, 
                    # print(f'player jersey #{player} DID NOT cross into offensive backfield')
                    pass
        
        
        # how many cross into the backfield?
        num_backfield = len(firsts)
        plays_wk.loc[(plays_wk['gameId'] == game) & (plays_wk['playId'] == play), 'numBackfield'] = num_backfield

        
        # who are they? and when did they cross (at what frame)?
        # firsts
        
        if num_backfield >= 5:
            plays_wk.loc[(plays_wk['gameId'] == game) & (plays_wk['playId'] == play), 'isBlitz'] = 1
        else:
            plays_wk.loc[(plays_wk['gameId'] == game) & (plays_wk['playId'] == play), 'isBlitz'] = 0


In [46]:
# save df with new columns to file
newfilename = 'week' + str(weekNum) + '_wBlitz.csv'

plays_wk.to_csv(newfilename)